In [2]:
import aiohttp
import asyncio
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings("ignore")

In [3]:
link_set = set()

In [6]:
semaphore = asyncio.Semaphore(50)

async def get_link(session,i) :
    url =  f"https://oto.com.vn/mua-ban-xe/p{i}"
    async with semaphore:
        try :
            await asyncio.sleep(0.5)
            async with session.get(url,timeout = 15) as resp :
                html = await resp.text()
                soup = BeautifulSoup(html , "html.parser")
                links_tag = soup.find_all("h3", class_="title")
                for tag in links_tag:
                    a_tag = tag.find('a')
                    if a_tag and 'href' in a_tag.attrs:
                        link = "https://oto.com.vn" + a_tag['href']
                        link_set.add(link)
                print(f"✅ Page {i} done.")
        except Exception as e:
            print(f"⚠️ Error at page {i}: {e}")

async def main():
    async with aiohttp.ClientSession() as session :
        tasks = [get_link(session, i) for i in range(1, 100)]
        await asyncio.gather(*tasks)
await main()
print(f"\n📦 Total link: {len(link_set)}")

✅ Page 49 done.
✅ Page 36 done.
✅ Page 14 done.
✅ Page 45 done.
✅ Page 50 done.
✅ Page 19 done.
✅ Page 28 done.
✅ Page 40 done.
✅ Page 20 done.
✅ Page 9 done.
✅ Page 32 done.
✅ Page 22 done.
✅ Page 48 done.
✅ Page 5 done.
✅ Page 13 done.
✅ Page 4 done.
✅ Page 27 done.
✅ Page 29 done.
✅ Page 42 done.
✅ Page 47 done.
✅ Page 43 done.
✅ Page 12 done.
✅ Page 23 done.
✅ Page 30 done.
✅ Page 8 done.
✅ Page 41 done.
✅ Page 35 done.
✅ Page 34 done.
✅ Page 17 done.
✅ Page 31 done.
✅ Page 39 done.
✅ Page 6 done.
✅ Page 10 done.
✅ Page 25 done.
✅ Page 21 done.
✅ Page 46 done.
✅ Page 26 done.
✅ Page 7 done.
✅ Page 2 done.
✅ Page 18 done.
✅ Page 37 done.
✅ Page 38 done.
✅ Page 15 done.
✅ Page 11 done.
✅ Page 44 done.
✅ Page 3 done.
✅ Page 1 done.
✅ Page 24 done.
✅ Page 16 done.
⚠️ Error at page 33: [WinError 10053] An established connection was aborted by the software in your host machine
✅ Page 51 done.
✅ Page 88 done.
✅ Page 66 done.
✅ Page 55 done.
✅ Page 52 done.
✅ Page 63 done.
✅ Page 74 done.


In [7]:
import aiohttp
import asyncio
from bs4 import BeautifulSoup

all_cars = []
semaphore = asyncio.Semaphore(50)  # Giới hạn tối đa 50 request đồng thời

async def detail(session, link):
    async with semaphore:
        try:
            async with session.get(link, timeout=20) as resp:
                if resp.status != 200:
                    print(f"❌ HTTP {resp.status} at {link}")
                    return

                html = await resp.text()
                soup = BeautifulSoup(html, "html.parser")
                car_info = {}

                # Tên xe
                name_tag = soup.find("div", class_="group-title-detail")
                name = name_tag.find("h1", class_="title-detail").get_text(strip=True) if name_tag else None
                car_info["Name"] = name

                # Giá xe
                price_tag = soup.find("span", class_="price")
                price = price_tag.get_text(strip=True) if price_tag else None
                car_info["Price"] = price

                # Các thông tin khác
                labels = soup.find_all("label", class_="label")
                for label in labels:
                    key = label.get_text(strip=True)
                    value_node = label.find_next_sibling(text=True)

                    if not value_node or value_node.strip() == "":
                        div = label.find_next("div")
                        value = div.get_text(strip=True) if div else "N/A"
                    else:
                        value = value_node.strip()

                    car_info[key] = value

                all_cars.append(car_info)
                print(f"✅ Done: {link}")

        except Exception as e:
            print(f"❌ ERROR at {link}: {e}")

async def main():
    async with aiohttp.ClientSession() as session:
        tasks = [detail(session, link) for link in link_set]
        await asyncio.gather(*tasks)

await main()

✅ Done: https://oto.com.vn/mua-ban-xe-kia-rondo-dong-nai/xe-zin-100-k-tai-nan-ngap-nuoc-lan-banh-71-van-bh-6-thang-sd-aidxc23338193
✅ Done: https://oto.com.vn/mua-ban-xe-mercedes-benz-glc-200-hcm/gia-1-ty-679-trieu-aidxc23327011
✅ Done: https://oto.com.vn/mua-ban-xe-ford-explorer-hcm/xe-chinh-hang-nhap-mybao-hanh-1-nam-aidxc23336261
✅ Done: https://oto.com.vn/mua-ban-xe-mercedes-benz-maybach-s450-hcm/mercedes-l-4matic-model-2023-aidxc23326982
✅ Done: https://oto.com.vn/mua-ban-xe-ford-ranger-ha-noi/xe-ban-tai-4x4-day-du-do-choi-va-do-zin-gia-cuc-tot-aidxc23337047
✅ Done: https://oto.com.vn/mua-ban-xe-nissan-x-trail-ha-noi/zin-5-van-km-aidxc23328760
✅ Done: https://oto.com.vn/mua-ban-xe-bmw-x7-hcm/co-hoi-so-huu-suv-hang-sang-voi-gia-tri-vuot-troi-aidxc23342470
✅ Done: https://oto.com.vn/mua-ban-xe-toyota-yaris-ha-noi/2010-13-at-100000-km-aidxc23346488
✅ Done: https://oto.com.vn/mua-ban-xe-ford-everest-hcm/doi-cao-di-luot-nuoc-son-con-rat-dep-aidxc23337714
✅ Done: https://oto.com.vn/mua-

CancelledError: 